In [119]:
import csv 
import re
novelists = []
with open('authors.csv','r') as file: 
    csv_file = csv.reader(file)
    for line in csv_file: 
        if len(line[0]) == 1: 
            continue
        else: 
            name = re.sub(f'\(.*?\)','',line[0])
            name = name.split(' ')
            novelists.append(f'{name[-1]}, {" ".join(name[:-1])}')
print(len(novelists))

356


In [153]:
# Text#,Type,Issued,Title,Language,Authors,Subjects
# 0     1    2      3     4        5,      6
found = {author: [] for author in novelists}
with open('pg_catalog.csv','r') as file:
    csv_file = csv.reader(file)
    next(csv_file)
    for line in csv_file:
        if len(line) < 6: 
            continue 
        if line[1] != 'Text': 
            continue
        if not re.search('fiction|Fiction', line[6]):
            continue
        if line[4] != 'en': 
            continue
        author = line[5]
        if 'Translator' in author or 'Contributor' in author: 
            continue # only want original works by one author 
        for name in novelists: 
            if re.search(name,author):
                found[name].append({'gutenberg_id':line[0],
                                'title':line[3],
                                'authors':line[5],
                                'subjects': line[6]    
                            })
                break

In [159]:
books = []
for list in found.values(): 
    books.extend(list)
print(len(books))

1288


In [83]:
'''
Code from this file: 
https://github.com/MengtingWan/goodreads/blob/master/samples.ipynb
'''
import json
import numpy as np
def load_data(file_name):
    data = []
    with open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            data.append(d)

    return data

authors = load_data('goodreads_book_authors.json')

In [155]:
gutenberg_authors = []
for author, texts in found.items(): 
    if len(texts) > 0:
        gutenberg_authors.append(author)

def format_name(name): 
    name = name.split(',')
    if re.search('\w{1}\. \w{1}\.',name[1]):
        initials = re.findall('\w{1}\.',name[1])
        name[1] = re.sub(f'{initials[0]} {initials[1]}',f'{initials[0]}{initials[1]}',name[1])
    if 'ë' in name[0]:
        name[0] = 'Bronte'
    full_name = f'{name[1].strip()} {name[0].strip()}'
    full_name = full_name.strip()
    return full_name

for idx, name in enumerate(gutenberg_authors): 
    gutenberg_authors[idx] = format_name(name)
print(len(gutenberg_authors))

97


In [156]:
found_goodreads = {}
for dict in authors: 
    author = dict["name"]
    for name in gutenberg_authors: 
        if name == author:
            found_goodreads[name] = dict
            break

In [158]:
mla = []

outfile = open('english_novelists.csv', 'w')
columns = [
    'gutenberg_name', 'goodreads_name', 'num_mla_entries', 'alt_name',
    'average_rating','author_id','text_reviews_count','ratings_count',
    'gutenberg_texts'       
]
writer = csv.DictWriter(outfile, columns)
writer.writeheader()
with open('gutenberg_novelists.csv','r') as file: 
    csv_file = csv.reader(file)
    next(csv_file)
    for line in csv_file: 
        name = format_name(line[0])
        if name in found_goodreads.keys(): 
            full_name = found[line[0]][0]['authors']
            if ';' in full_name: 
                full_name = full_name.split('; ')
                for entry in full_name: 
                    if line[0] in entry: 
                        full_name = entry 
                        break
            goodreads_info = found_goodreads[name]
            writer.writerow({
                'gutenberg_name': full_name,
                'goodreads_name': name, 
                'num_mla_entries': line[1],
                'alt_name': line[2],
                'average_rating':goodreads_info['average_rating'],
                'author_id':goodreads_info['author_id'],
                'text_reviews_count':goodreads_info['text_reviews_count'],
                'ratings_count':goodreads_info['ratings_count'],
                'gutenberg_texts': found[line[0]]       
            })
outfile.close()

In [135]:
# ! pip install gutenbergpy
import gutenbergpy.textget
gutenberg_id = 105 # Persuasion by Jane Austen
book = gutenbergpy.textget.get_text_by_id(gutenberg_id) 
book = gutenbergpy.textget.strip_headers(book) # remove headers
book = book.decode("utf-8") # turn bytes into string 